In [1]:
import mtt
from os.path import join, realpath
from os import getcwd
sbfile = join(realpath(getcwd()),'kinase_cascade_stage1.sb')
# sbfile = join(realpath(getcwd()),'checkpoint.xml')
wiring = mtt.Wiring.fromFile2(sbfile, {}, 1.)
wiring.blocks[1].ratC = 1.
wiring.connect(wiring.blocks[1].rv_up, wiring.blocks[0].Cprod)
wiring

In [2]:
model = mtt.MTT(wiring)
model.draw(3)

# Digitized current config

In [3]:
for block in wiring.blocks:
    print(block.getDigitizedParameterString(model))

# Simulation

In [4]:
sbml = model.toSBML()
from roadrunner import RoadRunner
xmod = RoadRunner(sbml)
xmod.reset()
xmod.simulate(0,1000,2000,selections=['time']+xmod.getFloatingSpeciesIds())
xmod.plot()

values = {s:xmod[s] for s in xmod.getFloatingSpeciesIds()}
print(values)

# SBML Simulation

In [9]:
# SBML comparison
import tellurium as te
with open(join(realpath(getcwd()),'kinase_cascade_stage1.sb')) as f:
    rr_model = te.loada(f.read())
rr_model.selections = ['time','__s0','__s1','__s6','__s4','__s7','__s8','__s2','__s9']
rr_model.simulate(0,1000,2000)

for var,value in values.items():
    print('rel diff for {}: {:.2f}'.format(var,(rr_model[var]-value)/max(rr_model[var],1.)))
    rr_model[var] = value

rr_model.plot()